Collecting the Response

In [44]:
import requests

url = "https://bulbapedia.bulbagarden.net/wiki/List_of_Pok%C3%A9mon_by_National_Pok%C3%A9dex_number"
response = requests.get(url)

if response.status_code == 200:
    print("Page loaded successfully!")
    html_content = response.text 
else:
    print(f"Failed to load the page. Status code: {response.status_code}")

with open('output.txt', 'w') as file:
    file.write(response.text)

Page loaded successfully!


Makeing the CSV from Request

In [45]:
import csv
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.text, "html.parser")
pokemon_data = []
pokemon_data.append(["#", "Name", "Type"])
rows = soup.find_all("tr", style="background:#FFF")
for index, row in enumerate(rows, start=1):
    title = row.find_all("a")[1].get("title").split(" ")[:-1]
    title = " ".join(title)
    title = title.lower()
    form = row.find("small")
    if form:
        form_text = form.get_text().strip()
        if "Alolan" in form_text:
            title += "_Alolan"
        elif "Galarian" in form_text:
            title += "_Galarian"
    types = row.find_all("a")[2:]
    types = [t.get("title").replace(" (type)", "") for t in types]
    types = ",".join(types)
    pokemon_data.append([index, title, types])
with open("pokemon_data.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerows(pokemon_data)

print("Data saved to pokemon_data.csv")

Data saved to pokemon_data.csv


IMAGES OF POKEMON

We downloaded it from kaggle and used the below code to map the Images to the CSV file we Previously Created, Cleaning the Data-Set with no Images

In [46]:
import os
import pandas as pd

folder_path = "Pokemon_Dataset"
df = pd.read_csv("pokemon_data.csv")
df["Image_ID"] = ""
df["Image_Path"] = ""

new_images = {}
for filename in os.listdir(folder_path):
    if filename.endswith(".png"):
        pokemon_name = filename[:-4]
        if pokemon_name in df["Name"].values:
            image_id = df.loc[df["Name"] == pokemon_name, "#"].values[0]
            new_image_name = f"{image_id}_{pokemon_name}.png"
            new_images[filename] = new_image_name
            df.loc[df["Name"] == pokemon_name, "Image_ID"] = new_image_name
            df.loc[df["Name"] == pokemon_name, "Image_Path"] = os.path.join(folder_path, new_image_name)
            os.rename(os.path.join(folder_path, filename), os.path.join(folder_path, new_image_name))
df = df[df["Image_ID"] != ""]
df.to_csv("pokemon_data.csv", index=False)